<a href="https://colab.research.google.com/github/AkSharma9604/AKsharma/blob/main/TextClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import pandas as pd

# Data for the models
data = {
    'Model': ['BERT', 'RoBERTa', 'GPT', 'DistilBERT', 'ALBERT'],
    'Accuracy': [0.92, 0.93, 0.88, 0.91, 0.90],
    'F1-Score': [0.91, 0.92, 0.85, 0.89, 0.88],
    'Inference Time (ms)': [120, 130, 150, 90, 110],
    'Memory Usage (MB)': [400, 450, 600, 250, 300],
    'Model Size (MB)': [450, 470, 1200, 200, 250]
}

# Create a DataFrame
df = pd.DataFrame(data)

# Criteria for minimization (Inference Time, Memory Usage, and Model Size)
criteria_min = ['Inference Time (ms)', 'Memory Usage (MB)', 'Model Size (MB)']

# Normalize the data
def normalize(df, criteria_min):
    normalized_df = df.copy()

    for column in df.columns[1:]:
        if column in criteria_min:
            # For minimization
            min_val = df[column].min()
            max_val = df[column].max()
            normalized_df[column] = (max_val - df[column]) / (max_val - min_val)
        else:
            # For maximization
            min_val = df[column].min()
            max_val = df[column].max()
            normalized_df[column] = (df[column] - min_val) / (max_val - min_val)

    return normalized_df

# Normalize the data
normalized_df = normalize(df, criteria_min)

# Weights for each criterion
weights = {
    'Accuracy': 0.4,
    'F1-Score': 0.3,
    'Inference Time (ms)': 0.2,
    'Memory Usage (MB)': 0.1,
    'Model Size (MB)': 0.1
}

# Calculate the weighted normalized decision matrix
weighted_normalized_df = normalized_df.copy()

for column in weighted_normalized_df.columns[1:]:
    weighted_normalized_df[column] = weighted_normalized_df[column] * weights[column]

# Calculate the ideal and negative-ideal solutions
ideal_solution = weighted_normalized_df.iloc[:, 1:].max()
negative_ideal_solution = weighted_normalized_df.iloc[:, 1:].min()

# Calculate the Euclidean distance from the ideal and negative-ideal solutions
def euclidean_distance(df, solution):
    return np.sqrt(((df - solution) ** 2).sum(axis=1))

distance_to_ideal = euclidean_distance(weighted_normalized_df.iloc[:, 1:], ideal_solution)
distance_to_negative_ideal = euclidean_distance(weighted_normalized_df.iloc[:, 1:], negative_ideal_solution)

# Calculate the relative closeness to the ideal solution
relative_closeness = distance_to_negative_ideal / (distance_to_ideal + distance_to_negative_ideal)

# Add the relative closeness to the dataframe
df['Relative Closeness'] = relative_closeness

# Sort the models by relative closeness (higher is better)
sorted_df = df.sort_values(by='Relative Closeness', ascending=False)

# Display the sorted results
print("Sorted Models based on TOPSIS Method:")
print(sorted_df[['Model', 'Relative Closeness']])


Sorted Models based on TOPSIS Method:
        Model  Relative Closeness
1     RoBERTa            0.776105
0        BERT            0.750565
3  DistilBERT            0.651307
4      ALBERT            0.477056
2         GPT            0.000000
